In [1]:
%load_ext gams_magic

In [2]:
%%gams

#Include data from .csv into GAMS
#Instructions:
#http://www.gams.com/help/index.jsp?topic=%2Fgams.doc%2Fuserguides%2Fuserguide%2F_u_g__set_definition.html
#https://www.gams.com/latest/docs/UG_DataExchange_ASCII.html
#In order to include the csv file, save the csv file in the same folder

Sets patients /1*290/,
     features /Diagnosis, Attr1, Attr2/;
table data(patients,features)
$ondelim
$include traindata.csv
$offdelim

display data;

scalar m number of patients /290/,
       mu user-specified parameter /0.5/;



display m;

#w1, w2, b: defines a linear function w1*attr1+w2*attr2+b
#This function will be used to classify maglinant (function value >0) versus benign (function value <0).

variables w1, w2, b;
free variable totalpenalty;
positive variables penalty(patients);

equations obj, measurepenalty(patients);

obj..
totalpenalty =e= sum(patients, penalty(patients))/m + mu*(w1*w1+w2*w2);

measurepenalty(patients)..
penalty(patients) =g=
      1- data(patients,'diagnosis')*(w1*data(patients,'attr1')+w2*data(patients,'attr2') + b);

model classification/all/;

#This controls the number of rows that are listed for each equation in the EQUATION LISTING section of the listing file
option limrow=100;
solve classification using qcp minimizing totalpenalty;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),0.2699,291,294,QCP,CONOPT,0.006


In [ ]:
%gams_lst

In [3]:
%gams display w1.l, w2.l, b.l;
%gams_lst -e

E x e c u t i o n


----    341 VARIABLE w1.L                  =        0.336  
            VARIABLE w2.L                  =        0.098  
            VARIABLE b.L                   =       -8.057  




In [ ]:
%gams_cleanup